In [1]:
# Import dependencies
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import json

In [ ]:
from sqlalchemy import create_engine

conn = create_engine(url_object).connect()

In [ ]:
from sqlalchemy import URL

url_object = URL.create(
    "postgresql",
    username="-----",
    password="-----",  
    host="------",
    database="-------",
)


In [2]:
# Get a dataframe with the recommended URIs to get the features for
pid_list=[0,5,9,11]
recommensations_df = pd.Series(pid_list, name='pid' )
recommensations_df=recommensations_df.to_frame()
recommensations_df.head()


,pid
0,0
1,5
2,9
3,11


In [4]:
# convert the dataframe to a dictionary then to a JSON string
recommendations_string=json.dumps(list(recommensations_df.T.to_dict().values()))       # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S

# open the file in write mode
output_path = "results.json"
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(recommendations_string)


 __To refresh the track_recommendations table in the Render database__

In [5]:
# Empty the track_recommendations table

load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()
cursor.execute("DELETE FROM pid_recommendations;")    # Deletes all the rows but keep the table
connection.commit()
cursor.close()
connection.close()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: fe_sendauth: no password supplied


In [11]:
# Repopulate the empty table from a json file
load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()

cursor.execute("set search_path to public")

with open(output_path) as file:
    data = file.read()

query_sql = """
INSERT INTO pid_recommendations SELECT * FROM
json_populate_recordset(NULL::pid_recommendations, %s);
"""

cursor.execute(query_sql, (data,))
connection.commit()

In [12]:




load_dotenv()
db_url = os.environ.get("link_render")

query="""
DROP VIEW IF EXISTS track_recommendation_pid_interim CASCADE;
CREATE VIEW track_recommendation_pid_interim AS 
SELECT playlist.pid, playlist.track_uri
FROM playlist
INNER JOIN pid_recommendations
ON playlist.pid = pid_recommendations.pid;


CREATE VIEW track_recommendation_pid AS 
SELECT track_recommendation_pid_interim.pid,track_recommendation_pid_interim.track_uri,
		track_features.danceability, track_features.energy, track_features.key, track_features.loudness, track_features.mode, track_features.speechiness,
       track_features.acousticness, track_features.instrumentalness, track_features.liveness, track_features.valence, track_features.tempo,
       track_features.duration_ms, track_features.time_signature, track_features.mode_minor, track_features.mode_major,
       track_features.key_none, track_features.key_0, track_features.key_1, track_features.key_2, track_features.key_3, track_features.key_4, track_features.key_5,
       track_features.key_6, track_features.key_7, track_features.key_8, track_features.key_9, track_features.key_10, track_features.key_11,
       track_features.time_signature_0, track_features.time_signature_1, track_features.time_signature_2,
       track_features.time_signature_3, track_features.time_signature_4, track_features.time_signature_5,
       track_features.time_signature_6, track_features.time_signature_7
FROM track_recommendation_pid_interim
INNER JOIN track_features
ON track_features.track_uri = track_recommendation_pid_interim.track_uri;

"""


engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )



In [13]:
load_dotenv()
db_url = os.environ.get("link_render")

query="""
SELECT * FROM track_recommendation_pid

"""


engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )

recommended_track_info_df = pd.DataFrame(results)
recommended_track_info_df

,pid,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,key_10,key_11,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7
0,5,spotify:track:78OGRRDepogg627XCOQ265,0.391,0.823,9,-7.125,1,0.0334,0.0170,0.001770,...,0,0,0,0,0,0,1,0,0,0
1,5,spotify:track:3cHyrEgdyYRjgJKSOiOtcS,0.581,0.963,11,-4.087,1,0.0981,0.0295,0.000000,...,0,1,0,0,0,0,1,0,0,0
2,5,spotify:track:3bidbhpOYeV4knp8AIu8Xn,0.641,0.922,2,-4.457,1,0.0786,0.0291,0.000000,...,0,0,0,0,0,0,1,0,0,0
3,5,spotify:track:7fYbFYt7X4FZvuJJC90EX0,0.654,0.797,1,-5.278,0,0.1820,0.0216,0.000010,...,0,0,0,0,0,0,1,0,0,0
4,5,spotify:track:3DmW6y7wTEYHJZlLo1r6XJ,0.699,0.529,2,-7.548,1,0.0487,0.0317,0.000036,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,11,spotify:track:3WS7spXVlbeC5kjePmHMQW,0.735,0.555,1,-8.840,0,0.0366,0.3820,0.856000,...,0,0,0,0,0,0,1,0,0,0
277,11,spotify:track:6xxkyxprEd4B9a28yrhsyR,0.802,0.770,5,-6.339,0,0.0407,0.0556,0.884000,...,0,0,0,0,0,0,1,0,0,0
278,11,spotify:track:4bNKvePgcSeo9eQM2yIbVE,0.607,0.595,7,-9.237,0,0.0542,0.1610,0.018800,...,0,0,0,0,0,0,1,0,0,0
279,11,spotify:track:5uIeRAC9O1ZXPQEEqpPVLL,0.920,0.364,10,-9.371,1,0.0381,0.0196,0.000298,...,1,0,0,0,0,0,1,0,0,0
